#### imports

In [ ]:
from ramanchada2.theoretical_lines.model_from_lines import model_from_lines
from ramanchada2.io.simulated.read_simulated_lines import read_simulated_lines
from ramanchada2.io.simulated.crystal.discrete_lines_dat import lines_from_crystal_dat
from ramanchada2 import spectrum

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
plt_fig_ax = lambda cols=1: plt.subplots(1, cols, figsize=(12, 5), tight_layout=True)

In [ ]:
vasp_sim_fn = '/data/RamanSimulated/vasp/snCAL_vasp_raman_ALL.dat'
exp_spe_fn = '/data/RamanSpe/ICV-CSIC/iRPlus/785 nm/nCAL/nCAL02_iRPlus785_Z050_100_2100ms.txt'

# Read nCAL experimental spectrum

In [ ]:
spe = spectrum.from_local_file(exp_spe_fn)
spe = spe.normalize()
fig, ax = plt_fig_ax()
ax.plot(spe.x, spe.y, label='Experimental')
ax.legend()

# Build theoretical model from VASP simulation data

```
# mode   freq(cm-1)    alpha       beta2      activity
001     1408.09865   0.0000465   1.9848711  13.8940975
002     1408.08970  -0.0000000   1.9845608  13.8919258
003     1377.52649  -0.0000929   0.0000004   0.0000034
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
029        0.03564   0.0000000   0.0000000   0.0000003
030        0.09163   0.0000000   0.0000000   0.0000000
```

In [ ]:
mod, par = model_from_lines(*read_simulated_lines(open(vasp_sim_fn), 'vasp', ['I_tot']), 'voigt')
par['x0'].set(min=-500, max=500, value=0)
par['x1'].set(min=1/5, max=5, value=1)
xsim = np.arange(3000)
ysim = mod.eval(params=par, x=xsim)

fig, ax = plt_fig_ax()
ax.plot(spe.x, spe.y, label='Experimental')
ax.plot(xsim, ysim/10, label='Simulation')
ax.legend()

# Fit model to the data

In [ ]:
resfit_spe = mod.fit(spe.y, params=par, x=spe.x)
fity = resfit_spe.eval(x=spe.x)

fig, ax = plt_fig_ax()
ax.plot(spe.x, spe.y, label='Experimental')
ax.plot(spe.x, fity, '--', label='Model Fit')
ax.legend()

# Try with poorly calibrated data

## Produce poorly calibrated data by scaling and offset

In [ ]:
scaled_spe = spe.scale_xaxis_fun(lambda x: (x+50)*.5)
scaled_spe.plot(ax=ax, label='scaled')
scaled_spe = scaled_spe.normalize()
fig, ax = plt_fig_ax()
spe.plot(ax=ax, label='original spectrum')
scaled_spe.plot(ax=ax, label='scaled spectrum')

In [ ]:
clear_spe = (scaled_spe-scaled_spe.moving_minimum(16))
clear_spe = clear_spe.normalize()
fig, ax = plt_fig_ax()
scaled_spe.plot(ax=ax, label='scaled spectrum')
clear_spe.plot(ax=ax, label='baseline subtracted')

## Try direct fit

In [ ]:
p0 = par.copy()
resfit_clear_spe_direct = mod.fit(clear_spe.y, params=p0, x=clear_spe.x)
fity = resfit_clear_spe_direct.eval(x=clear_spe.x)

fig, ax = plt_fig_ax()
ax.plot(clear_spe.x, clear_spe.y, label='scaled - baseline')
ax.plot(clear_spe.x, fity, '--', label='Direct fit')
ax.legend()

## Try broadening

- fitting delta-line model to delta-line data leads to delta lines in merit phase space
- broadening delta lines will lead to smoother merit phase space, so easier to calculate gradients
- use results from brodened data as an initial guess for the real fit

### Algorithm

In [ ]:
def convolve_fit(params, sigma=None):
    ppar = params.copy()
    if sigma:
        convol_spe = clear_spe.convolve('gaussian', sigma=sigma).normalize()
        ppar['sigma'].set(min=1, max=sigma*2, value=sigma)
    else:
        convol_spe = clear_spe.normalize()
        ppar['sigma'].set(min=1, max=30, value=4)

    fit_res = mod.fit(convol_spe.y, params=ppar, x=convol_spe.x)
    xplot = np.arange(-200, 3000)
    yinit = mod.eval(params=fit_res.init_params, x=xplot)
    yfit = mod.eval(params=fit_res.params, x=xplot)
    fig, ax = plt_fig_ax()
    ax.plot(convol_spe.x, convol_spe.y, label='Data')
    ax.plot(xplot, yinit, ':', lw=1, label='Initial guess')
    ax.plot(xplot, yfit, label='Best fit')
    ax.set_ylim(-0.1, 1.1)
    ax.set_xlim(-100, 1600)
    ax.legend()
    return fit_res

fig, ax = plt_fig_ax()
clear_spe.normalize().plot(ax=ax, label='Original data')
clear_spe.convolve('gaussian', sigma=40).normalize().plot(ax=ax, label='Convolved data')

### broaden experimental lines by convolution with gaussian($\sigma=40$)

In [ ]:
p0 = par.copy()
fitres_convolve40 = convolve_fit(p0, 40)

### broaden experimental lines by convolution with gaussian($\sigma=5$)

In [ ]:
fitres_convolve5 = convolve_fit(fitres_convolve40.params, 5)

### fit model to data w/o broadening

In [ ]:
fitres_convolve0 = convolve_fit(fitres_convolve5.params)

## Compare with the fit of unsclaed data

In [ ]:
pltx = np.arange(2000)
fity_orig = resfit_spe.eval(x=pltx)
fity_conv = fitres_convolve0.eval(x=(pltx+50)*.5)

fig, ax = plt_fig_ax()
ax.plot(fity_orig, '--', label='fit original spectrum')
ax.plot(fity_orig, '-.', label='fit scaled spectrum + same offset')
ax.legend()

# Moving minimum as a baseline filter

In [ ]:
from ramanchada2 import spectrum
import matplotlib.pyplot as plt
ss = spectrum.from_local_file('/data/RamanSpe/ICV-CSIC/iRPlus/785 nm/S0P/S0P02_iRPlus785_Z020_100_12000ms.txt')
ss0 = ss - ss.moving_minimum(80)
ss1 = ss - ss.moving_minimum(15)
fig, ax = plt.subplots(1, 2, figsize=(15, 5), tight_layout=True)
ss.plot(ax=ax[0], label='original')
ss0.plot(ax=ax[0], label='movmin(80)')
ss1.plot(ax=ax[0], label='movmin(15)')

ss.plot(ax=ax[1], label='original')
ss0.plot(ax=ax[1], label='movmin(80)')
ss1.plot(ax=ax[1], label='movmin(15)')
ax[1].set_ylim(0, 8000)